# Flatiron Phase 4 Project

* <b>Name:</b> James Benedito
* <b>Pace:</b> Part-Time
* <b>Instructor:</b> Morgan Jones

# Abstract

In this Jupyter notebook, I

# Business Problem

When it comes to business in general, it is important to make informed decisions on where to invest funds because a high ROI is imperative for a company to thrive. Data is a powerful tool that can be leveraged to determine where huge profits can potentially be gained in the future. 

This Jupyter notebook will examine time series data from Zillow to highlight the <b>Top 5 zipcodes in Clark County, Nevada</b> to invest in. Clark County is the largest district in Nevada state, encompassing all of Las Vegas, and is the fourth-largest district in the United States. My theoretical stakeholder is a real estate company located in Las Vegas, Nevada, who are seeking out the best locations for their clients. 

My suggestions to the real estate company will be informed by an <b>ARIMA model</b> with optimized parameters. This chosen ARIMA model will be used for <b>forecasting</b>, using previous ROI information to project potential ROI for various zipcodes in the Clark County area. From this <b>projected ROI</b> metric, I will construct my recommendations. 

# Dataset 

The data used for this project comes from Zillow and is stored in a file called <b>zillow_data.csv</b>. The dataset shows mean housing prices over time, with each row representing a specific zipcode in a particular US city. As mentioned previously, I will be focusing on merely a subset of this data. The dataframe will be filtered so that it only includes houses located in Nevada. 

# Data Exploration and Preprocessing

To start, I will explore the Zillow dataset to see how it is set up. From <b>.head()</b>, we see that the dataset has <b>272 columns</b>. Most of these columns represent dates in time and house the median price of houses for a particular zipcode. The other columns include <b>RegionID</b>, <b>RegionName</b>, <b>City</b>, <b>State</b>, <b>Metro</b>, <b>CountyName</b>, and <b>SizeRank</b>. We see that the data is quite expansive, encompassing information from many different cities and counties across the United States. As mentioned earlier, I want to filter the data so that it focuses merely on zipcodes in <b>Nevada</b> state because that is where the theoretical real estate company is based. From the main dataframe <b>zillow_df</b>, I will create a subset called <b>nv_df</b> that only has data on Nevada zipcodes. 

In [5]:
# import necessary packages
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)
import itertools
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
from matplotlib.pylab import rcParams
from statsmodels.tsa.stattools import adfuller

ImportError: cannot import name 'int' from 'numpy' (C:\Users\micha\anaconda3\envs\learn-env\lib\site-packages\numpy\__init__.py)

In [ ]:
# read dataset into pandas df
zillow_df = pd.read_csv('zillow_data.csv')
zillow_df.head()

In [ ]:
# filter dataset so it only includes zipcodes in Nevada 
nv_df = zillow_df[zillow_df['State']=='NV']
nv_df

Now that I have the filtered dataset, <b>nv_df</b>, I will explore the data further by looking at the <b>county</b> that has the most zipcodes. Using <b>.value_counts()</b>, I see that <b>Clark County</b> has the most zipcodes by far. I will therefore create another dataframe, <b>clark_df</b>, that focuses on the <b>61 zipcodes</b> in Clark County. I will also look at the distribution of Clark County zipcodes by city, once again employing the <b>.value_counts()</b> method. From <b>.value_counts()</b>, we see that <b>Las Vegas</b> is the city where most of the zipcodes are located in, accounting for <b>38 zipcodes</b> out of the 61 Clark County zipcodes.

In [ ]:
# looking at distribution of zipcodes amongst counties in Nevada
nv_df['CountyName'].value_counts()

In [ ]:
# creating dataframe for the zipcodes in Clark County
clark_df = nv_df[nv_df['CountyName']=='Clark']
clark_df

In [ ]:
# looking at distribution of zipcodes amongst cities in Clark County
clark_df['City'].value_counts()

One important thing that must be done to the <b>clark_df</b> prior to plotting it is to melt it. The dataframe right now is in wide form and must be converted to long form. The function <b>melt_data()</b> can be used to achieve this melting of <b>clark_df</b>. The returned dataframe will have <b>time</b> as the index and an aggregate value in the <b>value</b> column. From the <b>melt_data()</b> function, we see that the returned aggregate value is the <b>mean</b>. 

In [ ]:
# melt_data helper function provided in starter notebook
# converts dataframe from wide-form to long-form

def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [ ]:
melted_clark_df = melt_data(clark_df)
melted_clark_df

# Time Series Visualizations

Now that the dataframe has been melted, we can plot <b>melted_clark_df</b>. From the graph, we see that for all zipcodes in Clark County, there was an increase in price between 1996 and 2006. However, around the 2007 and 2008 mark, the prices began to dip. It was not until around 2013 that the housing prices started to really go up again.

In [ ]:
# plotting melted_clark_df
melted_clark_df.plot(figsize=(18,10), subplots=False, legend=True)
plt.show()

The plot above aggregates data from all the zipcodes into a single graph. I want to now look at the general trends for each of the 61 zipcodes in <b>clark_df</b> individually. From the graph below, we see that the pattern for each of the zipcodes is pretty similar. The 2007-2008 mark is when housing prices started to drop significantly. Then, around 2013, the housing prices started to really climb again.

In [ ]:
# Code adapted from Sanjit Varma 
# Source: https://github.com/sanjitva/Zillow-TimeSeries-Modeling/blob/main/final_notebook.ipynb

# plotting time series for all zipcodes in clark_df

# extracting zipcodes from clark_df
clark_zips = [a for a in clark_df['RegionName']]

# initialize empty dict
zip_dict = {}

# iterate over every zipcode in clark_zips
# use melt_data helper function to put data in long form
for zipcode in clark_zips:
    zip_dict[zipcode] = melt_data(clark_df[clark_df['RegionName']==zipcode])

# plot time series data
fig, ax = plt.subplots(figsize=(20,12))

for zipcode in zip_dict:
    ax.plot(zip_dict[zipcode],)  

ax.set_title('Zipcode Price Changes (1996-2018)', fontsize=20)
ax.set_xlabel('Year', fontsize=15)
ax.set_ylabel('Price (USD)', fontsize=15);

Prior to creating a time series model, I want to look at ROI since 2013. Given that the dataset goes on until 2018, this would be a <b>5-year ROI</b>. I will add a column to the <b>clark_df</b> that stores this calculated <b>5-year ROI</b>. Once I have the <b>5-year ROI</b> column made in the original <b>clark_df</b> dataframe, I will then make a new dataframe called <b>clark_df_filtered</b>, which only houses zipcodes that have an <b>ROI greater than 1.0 (100%)</b>. From there, I will plot the zipcodes in <b>clark_df_filtered</b> with their corresponding ROI values converted to a percentage. 

In [ ]:
# creating 5_yr_ROI col in clark_df
clark_df['5_yr_ROI'] = (clark_df['2018-04'] - clark_df['2013-04'])/(clark_df['2013-04'])
clark_df['5_yr_ROI']

In [ ]:
# creating filtered clark_df that only includes zipcodes that have 5_yr_ROI greater than 1.0 (100%)
clark_df_filtered = clark_df[clark_df['5_yr_ROI']>=1.0]
clark_df_filtered

In [ ]:
# Code adapted from Sanjit Varma 
# Source: https://github.com/sanjitva/Zillow-TimeSeries-Modeling/blob/main/final_notebook.ipynb

# plotting ROI for each zipcode

fig, ax = plt.subplots(figsize=(60,50))

x_labels = [str(a) for a in clark_df_filtered['RegionName']]
x = list(range(1,11))
y = [a for a in clark_df_filtered['5_yr_ROI']]

ax.bar(x,y)

ax.set_xticks(x)
ax.set_xticklabels(x_labels)
ax.set_yticks([a/10 for a in list(range(0,15,1))])
ax.set_yticklabels([str(a*10)+'%' for a in list(range(0,15,1))])
ax.set_ylabel('Growth (%)', fontsize='30')
ax.set_xlabel('Zipcodes', fontsize='30')
ax.set_title('Avg % Return on Investment',fontsize='40');

The zipcode with the greatest ROI is <b>89104</b>, which has an <b>ROI of about 130%</b>. We can utilize the data for this zipcode to create our initial time series model. 

# Data Preparation: 89104 Zipcode

Prior to modeling, we need to prepare our data for the <b>89104</b> zipcode. I will begin by making a new dataframe that houses the 89104 zipcode data. From there, I will employ the <b>melt_data_2()</b> function to convert the dataframe from wide form to long form. The <b>melt_data_2()</b> function is set up exactly like the <b>melt_data()</b> helper function. The only difference is that <b>melt_data_2()</b> factors in the new <b>5_yr_ROI</b> column that was added earlier.

In [ ]:
# same as melt_data helper function 
# just added '5_yr_ROI' under id_vars

def melt_data_2(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName', '5_yr_ROI'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [ ]:
# make new df housing only 89104 zipcode data, then use melt_data to convert to long form
zipcode_89104_data = clark_df_filtered[clark_df_filtered['RegionName']==89104]
tseries_89104 = melt_data_2(zipcode_89104_data)
tseries_89104

Now that we have the melted <b>tseries_89104</b> data, our next step prior to modeling is checking for <b>seasonality</b> and <b>trends</b>. I plan to build an <b>ARIMA model</b>. Therefore, the assumptions of the <b>ARIMA model</b> are that the time series data is <b>non-seasonal</b> and <b>detrended</b>. If these assumptions are not met, we need to do more data preparation prior to building our model. 

# Seasonality and Trends

To check for trends and seasonality, we need to use methods that allow us to understand a time series' <b>stationarity</b>. In order for the assumptions of <b>non-seasonality</b> and <b>detrending</b> to be met, the data must be deemed <b>non-stationary</b>. Two ways to determine stationarity are <b>rolling statistics</b> and the <b>Dickey-Fuller Test</b>. I plan to use both of these methods on <b>tseries_89104</b>. 

In [ ]:
# plotting tseries_89104 to visualize it
tseries_89104.plot(figsize=(12,6), linewidth=2, fontsize=10);

In [ ]:
# check rolling statistics

# determine rolling statistics
roll_mean = tseries_89104.rolling(window=12, center=False).mean()
roll_std = tseries_89104.rolling(window=12, center=False).std()

# plot rolling statistics
fig = plt.figure(figsize=(12,6))
plt.plot(tseries_89104, color='blue',label='Original')
plt.plot(roll_mean, color='red', label='Rolling Mean')
plt.plot(roll_std, color='black', label = 'Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation for 89104 Zipcode')
plt.show()

In [ ]:
# perform Dickey-Fuller test
print ('Results of Dickey-Fuller Test: \n')
dftest = adfuller(tseries_89104['value'])

# extract and display test results
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

From our <b>rolling statistics</b> analysis, we see that the <b>rolling mean</b> is <b>not constant over time</b>, thus indicating <b>non-stationarity</b>. This <b>non-stationary</b> behavior is further supported by the <b>Dickey-Fuller Test</b> results, whose test statistic has an insignificant <b>p-value greater than 0.05</b>. This means that the null hypothesis, which states that the <b>time series is non-stationary</b> cannot be rejected. Therefore, more processing of the data must be done before getting into modeling. 

# Removing Seasonality and Trends

There are numerous methods that can be employed to remove seasonality and trends in a time series. The methods I want to use for <b>tseries_89104</b> are <b>subtracting rolling mean</b>, <b>subtracting weighted rolling mean</b>, <b>log transform</b> and <b>square root transform</b>. I will create visualizations for the time series when all four of these strategies are applied. I will also perform the <b>Dickey Fuller Test</b> for these methods to confirm non-seasonality. 

In [ ]:
# subtract rolling mean
tseries_89104_minus_roll_mean = tseries_89104 - roll_mean
tseries_89104_minus_roll_mean.dropna(inplace=True)

fig = plt.figure(figsize=(11,7))
plt.plot(tseries_89104_minus_roll_mean, color='blue', label='value - rolling mean')
plt.legend(loc='best')
plt.title('Zipcode 89104 Data (Subtract Rolling Mean)')
plt.show(block=False)

In [ ]:
# perform Dickey-Fuller test for rolling mean results
print ('Results of Dickey-Fuller Test: \n')
dftest = adfuller(tseries_89104_minus_roll_mean['value'])

# extract and display test results
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

In [ ]:
# weighted rolling mean

# use Pandas ewm() to calculate Exponential Weighted Moving Average
exp_roll_mean = tseries_89104.ewm(halflife=2).mean()

# subtract the moving average from the original data
tseries_89104_minus_exp_roll_mean = tseries_89104 - exp_roll_mean

# plot
fig = plt.figure(figsize=(11,7))
plt.plot(tseries_89104_minus_exp_roll_mean, color='blue', label='value - rolling mean')
plt.legend(loc='best')
plt.title('Zipcode 89104 Data (Subtract Weighted Rolling Mean)')
plt.show(block=False)

In [ ]:
# perform Dickey-Fuller test for weighted rolling mean results
print ('Results of Dickey-Fuller Test: \n')
dftest = adfuller(tseries_89104_minus_exp_roll_mean['value'])

# extract and display test results
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

In [ ]:
# plot a log transform
tseries_89104_log = np.log(tseries_89104)
fig = plt.figure(figsize=(12,6))
plt.plot(tseries_89104_log, color='blue');

In [ ]:
# perform Dickey-Fuller test for log transform results
print ('Results of Dickey-Fuller Test: \n')
dftest = adfuller(tseries_89104_log['value'])

# extract and display test results
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

In [ ]:
# plot a square root transform
tseries_89104_sqrt = np.sqrt(tseries_89104)
fig = plt.figure(figsize=(12,6))
plt.plot(tseries_89104_sqrt, color='blue');

In [ ]:
# perform Dickey-Fuller test for square root transform results
print ('Results of Dickey-Fuller Test: \n')
dftest = adfuller(tseries_89104_sqrt['value'])

# extract and display test results
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

After applying all four methods, we see that the best results are yielded by the <b>log transformed</b> and <b>square root transformed</b> time series. The <b>p-values</b> for the <b>Dickey-Fuller Tests</b> for these methods are <b>about 0.01</b>, indicating significance at a threshold of 0.05 and <b>non-seasonality</b>. When <b>subtracting the rolling mean</b> and <b>the weighted rolling mean</b>, the <b>Dickey-Fuller Tests</b> have <b>p-values</b> that are <b>insignificant</b>, being about <b>0.24</b> and <b>0.12</b>, respectively. I will proceed with <b>tseries_89104_log</b> and <b>tseries_89104_sqrt</b> use them for my <b>ARIMA modeling</b>. Based on the model results, I will choose the one that I want to utilize for forecasting. 

# ARIMA Modeling

Now that the data is <b>non-seasonal</b> and <b>detrended</b>, we can begin the iterative modeling process. Remember, we're using <b>tseries_89104_log</b> and <b>tseries_89104_sqrt</b>. Let's go ahead and split <b>tseries_89104_log</b> and <b>tseries_89104_sqrt</b> into training and test sets. The training sets will house data from April 2011 to April 2016, while the test sets will be information from May 2016 to April 2018, which is where the dataset ends.

In [ ]:
# split tseries_89104_log and tseries_89104_sqrt into training and test sets
tseries_89104_log_train = tseries_89104_log['2011-04':'2016-04'] 
tseries_89104_log_test = tseries_89104_log['2016-05':] 
tseries_89104_sqrt_train = tseries_89104_sqrt['2011-04':'2016-04']
tseries_89104_sqrt_test = tseries_89104_sqrt['2016-05':]

Let's now turn our attention to creating our models, which we will fit on <b>tseries_89104_log_train</b> and <b>tseries_89104_sqrt_train</b>. An <b>ARIMA model</b> can be fit on time series data to predict future points. The parameters for ARIMA are specified as <b>(p,d,q)</b>, where p is the autoregressive part of the model, d is the integrated component, and q is the moving average portion of the model. (p,d,q) represents the model's <b>order</b>. In order to find the <b>optimal parameters</b> for (p,d,q), a <b>grid search method</b> can be employed. 

In [ ]:
# Setting up grid search

# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

# ARIMA Modeling: tseries_89104_log_train

In [ ]:
# Run a grid with pdq and seasonal pdq parameters and get the best AIC value
# For tseries_89104_log_train
ans_log = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(tseries_89104_log_train,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans_log.append([comb, combs, output.aic])
            print('ARIMA {} x {}: AIC Calculated={}'.format(comb, combs, output.aic))
        except:
            continue

In [ ]:
# Find the parameters with minimal AIC value for tseries_89104_log_train
ans_df_log = pd.DataFrame(ans_log, columns=['pdq', 'pdqs', 'aic'])
ans_df_log.loc[ans_df_log['aic'].idxmin()]

Based on the numbers for <b>pdq</b> and <b>pdqs</b> selected by our grid search, we can now plug in these optimal value combinations for our <b>order</b> and <b>seasonal_order</b> parameters. Let's run the model again and look at our results. We will also look at different visualizations using the <b>.plot_diagnostics()</b> method. 

In [ ]:
# Plug the optimal parameter values into a new SARIMAX model
# For tseries_89104_log_train
ARIMA_MODEL = sm.tsa.statespace.SARIMAX(tseries_89104_log_train, 
                                        order=(1, 1, 1), 
                                        seasonal_order=(0, 0, 0, 12), 
                                        enforce_stationarity=False, 
                                        enforce_invertibility=False)

# Fit the model and print results
output_log = ARIMA_MODEL.fit()

print(output_log.summary().tables[1])

In [ ]:
# Call plot_diagnostics() on tseries_89104_log_train
output_log.plot_diagnostics(figsize=(15, 18))
plt.show()

From the ARIMA model summary, we see that all coefficients are significant. From the visualizations generated by <b>.plot_diagnostics()</b>, we notice that the <b>KDE follows a relatively normal distribution</b> and matches up closely with the N(0,1) standard normal distribution curve. Though some of the points fall off the red guidance line, the <b>qq-plot</b> has dots that show a relatively linear trend. This indicates that the residuals are normally distributed. The top left plot shows no obvious seasonality; this is further supported by the <b>correlogram</b> on the bottom right, which tells us that the residuals in the time series have low correlations with the lagged versions of itself. 

# ARIMA Modeling: tseries_89104_sqrt_train

Similar to what we did with <b>tseries_89104_log_train</b>, we will find the optimal parameters for pdq and seasonal pdq for <b>tseries_89104_sqrt_train</b> and fit an optimized ARIMA model on it. 

In [ ]:
# Run a grid with pdq and seasonal pdq parameters and get the best AIC value
# For tseries_89104_sqrt_train
ans_sqrt = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(tseries_89104_sqrt_train,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans_sqrt.append([comb, combs, output.aic])
            print('ARIMA {} x {}: AIC Calculated={}'.format(comb, combs, output.aic))
        except:
            continue

In [ ]:
# Find the parameters with minimal AIC value for tseries_89104_sqrt_train
ans_df_sqrt = pd.DataFrame(ans_sqrt, columns=['pdq', 'pdqs', 'aic'])
ans_df_sqrt.loc[ans_df_sqrt['aic'].idxmin()]

Let's run the ARIMA model again with the grid search-selected parameters and look at our results. We will print out the model table summary and employ <b>plot_diagnostics()</b>, similar to what we did with the log times series. 

In [ ]:
# Plug the optimal parameter values into a new SARIMAX model
# For tseries_89104_sqrt_train
ARIMA_MODEL = sm.tsa.statespace.SARIMAX(tseries_89104_sqrt_train, 
                                        order=(1, 1, 0), 
                                        seasonal_order=(1, 1, 0, 12), 
                                        enforce_stationarity=False, 
                                        enforce_invertibility=False)

# Fit the model and print results
output_sqrt = ARIMA_MODEL.fit()

print(output_sqrt.summary().tables[1])

In [ ]:
# Call plot_diagnostics() on tseries_89104_sqrt_train
output_sqrt.plot_diagnostics(figsize=(15, 18))
plt.show()

From the ARIMA model summary, we see that all of the coefficients are significant. From the visualizations generated by <b>.plot_diagnostics()</b>, we notice that the <b>KDE follows a relatively normal distribution</b> and matches up well with the N(0,1) curve; however, there is a slight concave portion at the top. While some of the points fall off the red guidance line in the <b>qq-plot</b>, the blue dots still show a relatively linear trend. This indicates that the residuals are normally distributed. The top left plot shows no obvious seasonality; this is also supported by the <b>correlogram</b>, which tells us that the residuals in the time series have low correlations with the lagged versions of itself. 

# ARIMA Modeling: tseries_89104

Earlier, we said that the original <b>tseries_89104</b> data exhibited some seasonality based on the Dickey-Fuller test results, which yielded an insignificant p-value of <b>about 0.09</b>. However, for comparison purposes, let's also create an ARIMA model for the raw <b>tseries_89104</b> time series. Similar to the modeling done for the previous two time series, <b>tseries_89104_log_train</b> and <b>tseries_89104_sqrt_train</b>, we will use a grid search to determine the optimal parameters. It should also be noted that we will fit the model on <b>tseries_89104_train</b>, which houses data between April 2011 and April 2016. The test set will be data from May 2016 to April 2018, just like how we set it up for the log transformed and square root transformed data.

In [ ]:
# split tseries_89104 into training and test set
tseries_89104_train = tseries_89104['2011-04':'2016-04']
tseries_89104_test = tseries_89104['2016-05':]

In [ ]:
# Run a grid with pdq and seasonal pdq parameters and get the best AIC value
# For tseries_89104_train
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(tseries_89104_train,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
            print('ARIMA {} x {}: AIC Calculated={}'.format(comb, combs, output.aic))
        except:
            continue

In [ ]:
# Find the parameters with minimal AIC value for tseries_89104_train
ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
ans_df.loc[ans_df['aic'].idxmin()]

In [ ]:
# Plug the optimal parameter values into a new SARIMAX model 
# For tseries_89104_train
ARIMA_MODEL = sm.tsa.statespace.SARIMAX(tseries_89104_train, 
                                        order=(1, 1, 0), 
                                        seasonal_order=(1, 1, 0, 12), 
                                        enforce_stationarity=False, 
                                        enforce_invertibility=False)

# Fit the model and print results
output = ARIMA_MODEL.fit()

print(output.summary().tables[1])

In [ ]:
# Call plot_diagnostics() on tseries_89104_train
output.plot_diagnostics(figsize=(15, 18))
plt.show()

From the ARIMA model summary, we see that all of the coefficients are significant. From the visualizations generated by <b>.plot_diagnostics()</b>, we notice that the <b>KDE follows a relatively normal distribution</b> and matches up closely with the N(0,1) curve. While some of the points fall off the red guidance line in the <b>qq-plot</b>, the blue dots still show a relatively linear trend, indicating that the residuals are normally distributed. The top left plot shows no obvious seasonality; this is also supported by the <b>correlogram</b>, which tells us that the residuals in the time series have low correlations with the lagged versions of itself. 

# Choosing Model

Between the three ARIMA models for <b>tseries_89104_log_train</b>, <b>tseries_89104_sqrt_train</b>, and <b>tseries_89104_train</b>, I will proceed with the model for <b>tseries_89104_train</b>.

# Validating Model 

Now that we've deemed the fit of the model to be satisfactory, we can validate the chosen model using predicted values to compare to real values in the time series. This will give us insight on the accuracy of our forecasts. I will employ the <b>get_prediction()</b> and <b>conf_int()</b> methods to achieve this model validation. In terms of forecasting, I will use <b>one-step ahead forecasting</b>. The metric I will utilize to determine the accuracy of the forecasts will be <b>MSE</b>. 

# One-Step Ahead Forecasting

In [ ]:
# Get predictions starting from 2016-05 and calculate confidence intervals
pred = output.get_prediction(start=pd.to_datetime('2016-05'), end=pd.to_datetime('2018-04'), dynamic=False)
pred_conf = pred.conf_int()
pred_conf

In [ ]:
# Plot real vs predicted values along with confidence interval

rcParams['figure.figsize'] = 15, 6

# Plot observed values
ax = tseries_89104['2014':].plot(label='observed')

# Plot predicted values
pred.predicted_mean.plot(ax=ax, label='One-Step Ahead Forecast', alpha=0.9)

# Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='g', alpha=0.5, label='Confidence Interval')

# Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('Price')
plt.legend()

plt.show()

Based on model validation using the test set, we see that the confidence intervals get bigger the farther we go along the time series. This is an indication that the model has room for improvement. However, for this particular project, we will proceed with it. 

In [ ]:
#from sklearn.metrics import mean_absolute_error

#mae = mean_absolute_error(actual_values, predicted_values)
#print(mae)

# Forecasting Future (Zipcode 89104)

Now that we have tested our model, we can fit it onto the entire <b>tseries_89104</b> dataset.

In [ ]:
# fit entire tseries_89104
# use same order and seasonal order parameters
ARIMA_MODEL = sm.tsa.statespace.SARIMAX(tseries_89104, 
                                        order=(1, 1, 0), 
                                        seasonal_order=(1, 1, 0, 12), 
                                        enforce_stationarity=False, 
                                        enforce_invertibility=False)

# fit model and print results
output_tseries_89104 = ARIMA_MODEL.fit()

print(output_tseries_89104.summary().tables[1])

In [ ]:
# getting a forecast for the next 5 years (60 months) after April 2018 (last record in time series data)
forecast = output_tseries_89104.get_forecast(60)
future_prediction = forecast.conf_int()
future_prediction['value'] = forecast.predicted_mean
future_prediction.columns = ['lower','upper','prediction'] 
future_prediction

In [ ]:
# plotting our forecast

fig, ax = plt.subplots()
tseries_89104.plot(ax=ax, label='Real Values')


future_prediction['prediction'].plot(ax=ax, label='predicted value', ls='--')

ax.fill_between(x=future_prediction.index, 
                y1=future_prediction['lower'], 
                y2=future_prediction['upper'], 
                color='lightpink',
                label='Confidence Interval')
ax.legend() 
plt.ylabel("Average Price")
plt.title("Forecasted House Prices (Zipcode 89104)")
plt.show()

# Forecasting Future (All Zipcodes)

Just like we used our model to forecast 89104 zipcode prices, we can now apply this model that we created to all the zipcodes in <b>clark_df_filtered</b>. Recall that this dataframe includes zipcodes in Clark County that had a 5-year ROI of 1.0 or greater. Using a function, <b>forecast_model()</b>, which has the <b>order</b> and <b>seasonal order</b> parameters set to the ones used in the final ARIMA model, we can apply the ARIMA model we made earlier

In [ ]:
# extracting zipcodes from clark_df
clark_zips = [a for a in clark_df_filtered['RegionName']]

# initialize empty dict
zip_dict = {}

# iterate over every zipcode in clark_zips
# use melt_data_2 function to put data in long form
for zipcode in clark_zips:
    zip_dict[zipcode] = melt_data_2(clark_df_filtered[clark_df_filtered['RegionName']==zipcode])

In [ ]:
zip_dict

In [ ]:
# Code adapted from Fernando Aguilar
# Source: https://medium.com/@feraguilari/time-series-analysis-modfinalproyect-b9fb23c28309

def forecast_model(df, pdq=(1, 1, 0), pdqs=(1, 1, 0, 12), display=True, zc='input zipcode'):
    model = sm.tsa.statespace.SARIMAX(df, order=pdq, seasonal_order=pdqs)
    model_fit = model.fit()
    output = model_fit.get_prediction(start='2018-04', end='2023-04', dynamic=True)
    forecast_ci = output.conf_int()
    if display:
        fig, ax = plt.subplots(figsize=(13,6))
        output.predicted_mean.plot(label='Forecast')
        ax.fill_between(forecast_ci.index,forecast_ci.iloc[:, 0],forecast_ci.iloc[:, 1],
                        color='k', alpha=.25,label='Conf Interval')
        plt.title('Forecast of Monthly Returns')
        plt.xlabel('Time')
        plt.legend(loc='best')
        plt.show()
    year_1= (1+output.predicted_mean[:12]).prod()-1
    year_3=(1+output.predicted_mean[:36]).prod()-1
    year_5= (1+output.predicted_mean[:60]).prod()-1
    print(f'Total expected return in 1 year: {round(year_1*100,2)}%')
    print(f'Total expected return in 3 years: {round(year_3*100,2)}%')
    print(f'Total expected return in 5 year: {round(year_5*100,2)}%')
    tot_ret = [zc, year_1, year_3, year_5]
    return tot_ret

In [ ]:
forecast_model(tseries_89104)

# Recommendations

Find Top 5 zipcodes

# Limitations and Improvements

The final model used for forecasting had many issues (wide confidence intervals). Can only be used to project small amounts of time (still useful in this sense). 

Model for determining best zipcode can be improved 

Recommendations still solid because the zipcodes are in the Top 10 of ROI from 2013-2018 (assuming continuing trend).